In [39]:
#0 : Promotional  5.33% 折扣、下殺 特價。。。
#1: Engagement  31.56%   按讚、打卡 分享。。。
#2: Product Awareness  28% (產品) “new product” 系列、首發、新登場、開箱、推薦 新一代 進化版
#3: Brand Awareness 8.44%（品牌）
#Corporate Social Responsibility 5.33%
#Customer Service 3.11% (粉絲) 服務 品管
#Seasonal 18.67%     開學、情人節


In [70]:
import collections

In [772]:
from pymongo import MongoClient
import json
import os.path
from numpy import array

In [35]:
client = MongoClient('mongodb://localhost:27017/')

In [36]:
db = client['fanpage_research']
coll_fanpage = db['fanpage']
coll_post = db['post']
coll_post_detail = db['post_detail']

In [500]:
import numpy as np
import pandas as pd

In [1084]:
import jieba
import jieba.analyse

In [37]:
with open('fid_list_4.json') as d:
    all_fid_list = json.load(d)

In [1283]:
jieba.set_dictionary("dict.txt.big")
stop_words_path = "data/household_stop_words.txt"
jieba.analyse.set_stop_words(stop_words_path)

In [51]:
def pp(o):
    print json.dumps(o,indent=1,ensure_ascii=False)

In [40]:
def get_combined_message_by_label(X_array, y_array, label):
    nd_X = np.array(X_array)
    nd_y = np.array(y_array)
    words = ""
    for p in nd_X[nd_y==label]:
        if p['message'] is not None:
            words = words + p['message']
    return words

In [346]:
def get_keyword(X_array, y_array, label):
    combined_message = get_combined_message_by_label(X_array, y_array, label)
    return jieba.analyse.extract_tags(combined_message, 20)

### -----------------above functions are global functions----------

In [1487]:
class industry_data:
    def __init__(self,industry_name_list,all_fid_list):
        self.name_list = industry_name_list
        self.name_abbr = None
        self.all_fid_list = all_fid_list
        self.fid_list = None
        self.posts = None
        self.train_post = None
        self.train_label = None
        self.test_post = None
        self.test_label = None
    def get_name(self):
        name = ""
        if self.name_abbr == None:
            for i in self.name_list:
                name = name + i.rsplit(' ')[0]
            self.name_abbr = name
        return self.name_abbr
    def get_fid_list(self,refresh=False):
        if self.fid_list == None or refresh == True:
            fid_list = []
            for f in self.all_fid_list:
                if f['category'] in self.name_list:
                    fid_list.append(f['fid'])
            self.fid_list = fid_list
        return self.fid_list
    def get_posts(self,refresh=False):
        fid_list = self.get_fid_list()
        if self.posts == None or refresh == True:
            posts = []
            for f in fid_list:
                posts = posts + [p for p in coll_post.find({"fid":f},{'date':False,'_id':False})]
            self.posts = posts
        return self.posts
    def get_train_post_label(self,sample_number=50,refresh=False):
        self.train_label == None
        if self.train_post == None or self.train_label == None or refresh == True:
            file_name = 'data/train_data_'+self.get_name()+'.json'
            if refresh == True or (not os.path.isfile(file_name)):
                tmp_dict = {} 
                posts = self.get_posts()
                tmp_dict['train_X'] = [p for p in np.random.choice(posts, 50, replace=False)]
                tmp_dict['train_y'] = []
                with open(file_name,'w') as outfile:
                    json.dump(tmp_dict,outfile)
            with open(file_name,'r') as infile:
                j_file = json.load(infile)
            try:
                self.train_post = j_file['train_X']
                self.train_label = j_file['train_y']
            except:
                print "error occured when accessing"+file_name+"[train_X] or [train_y]"          
        return self.train_post,self.train_label
    def get_test_post_label(self, sample_number = 50, refresh=False):
        self.test_label == None
        if self.test_post == None or self.test_label == None or refresh == True:
            file_name = 'data/test_data_'+self.get_name()+'.json'
            if refresh == True or (not os.path.isfile(file_name)):
                tmp_dict = {} 
                train_X, train_y = self.get_train_post_label()
                all_posts = self.get_posts()
                posts_without_trainX = [i for i in all_posts if i not in train_X]
                tmp_dict['test_X'] = [p for p in np.random.choice(posts_without_trainX, 50, replace=False)]
                tmp_dict['test_y'] = []
                with open(file_name,'w') as outfile:
                    json.dump(tmp_dict,outfile)
            with open(file_name,'r') as infile:
                j_file = json.load(infile)
            try:
                self.test_post = j_file['test_X']
                self.test_label = j_file['test_y']
            except:
                print "error occured when accessing"+file_name+"[test_X] or [test_y]"          
        return self.test_post,self.test_label    
    def get_keyword_category(self):
        X, y = self.get_train_post_label()
        keyword_dict = collections.OrderedDict()
        if len(X)==len(y) and len(X)!=0:
            for i in range(4):
                combined_message = get_combined_message_by_label(X, y, i)
                keyword_dict['class_'+str(i)] = jieba.analyse.extract_tags(combined_message, 10)
        return keyword_dict

In [1685]:
class Classifier:
    def __init__(self,industry):
        self.name = industry
        self.keyword_dict = None
    def get_keyword_dict(self,refresh=False):
        if self.keyword_dict == None or refresh == True:
            file_name = 'data/keyword_'+self.name+'.json'
            if os.path.isfile(file_name):
                with open(file_name,'r') as infile:
                    keyword_file = collections.OrderedDict(json.load(infile))
                self.keyword_dict = keyword_file
            else:
                print file_name+" doesn't exist"
                return 
        return self.keyword_dict
    
# Below is stupid model
#     def _predict(self,message):
#         message_jieba = jieba.analyse.extract_tags(message,0)
#         key_dict = self.get_keyword_dict()
        
#         print 'message jieba : ',','.join(message_jieba)
#         print 'keyword 0',','.join(key_dict['class_0'])
#         print 'keyword 1 : ',','.join(key_dict['class_1'])
#         print 'keyword 2',','.join(key_dict['class_2'])
        
#         match_class_0 = list(set(message_jieba).intersection(key_dict['class_0']))
#         match_class_1 = list(set(message_jieba).intersection(key_dict['class_1']))
#         match_class_2 = list(set(message_jieba).intersection(key_dict['class_2']))

#         print "0: ", ','.join(match_class_0)
#         print "1: ", ','.join(match_class_1)
#         print "2: ", ','.join(match_class_2)
#         if len(match_class_1)>0:
#             if len(match_class_1)==1 and u'分享' in match_class_1:
#                 if u'粉絲' in message_jieba:   #粉絲＋分享 --> 1
#                     return 1
#                 else:
#                     if len(match_class_0)>0:  #分享＋送好禮-->1
#                         return 1                    
#                     return 3
#             return 1
#         elif len(match_class_0)>0:
#             return 0
#         elif len(match_class_2)>0:
#             return 2
#         else:
#             return 3
#     def predict(self,data_set):
#         pre_y = []
#         predict_class = None
#         for d in data_set:
#             if d['message']!=None:
#                 pre_y.append(self._predict(d['message']))
#                 predict_class = self._predict(d['message'])
#             else:
#                 if d['post_type'] in ['album','photo','cover_photo','video','video_inline','image_share','animated_image_share','video_direct_response']:
#                     pre_y.append(1)
#                     predict_class = 1
#                 else:
#                     pre_y.append(3)
#                     predict_class = 3
#             print predict_class
#             print d
#             print '---------------------------'
#         return pre_y
#Above is stupid model

    def _make_ML_X(self,input_X,print_out=False):
        key_dict = self.get_keyword_dict()
        ML_X = []
        for obj in input_X:
            tmp = []
            has_vivid = 0
            if obj['post_type'] in ['album','photo','cover_photo','video','video_inline','image_share','animated_image_share','video_direct_response']:
                has_vivid = 1
            match = []
            obj_message = obj['message']
            for k in key_dict:
                if obj_message!=None:
                    message_jieba = jieba.analyse.extract_tags(obj['message'],0)
                    match = list(set(message_jieba).intersection(key_dict[k]))
                if print_out:
                    print 'match '+k+' : ',','.join(match)
                tmp.append(len(match))
            tmp.append(has_vivid)
            ML_X.append(tmp)
        return ML_X

# Machine Learning Time

In [1006]:
from sklearn import tree

In [1011]:
from sklearn.linear_model import LogisticRegression

In [1700]:
class ML:
    # industry_name is a list obj
    def __init__(self,industry_name_list):
        self.industry_obj = industry_data(industry_name_list,all_fid_list)
        self.train_X, self.train_y = self.industry_obj.get_train_post_label()
        self.test_X, self.test_y = self.industry_obj.get_test_post_label()
        self.classifier = Classifier(self.industry_obj.name_abbr)
        self.ML_train_X = self.classifier._make_ML_X(self.train_X)
        self.ML_test_X = self.classifier._make_ML_X(self.test_X)
        self.model_lg = None
        self.model_tree = None
    def get_model_tree(self,refresh=False):
        if self.model_tree == None or refresh == True:
            clf = tree.DecisionTreeClassifier()
            clf_model = clf.fit(np.array(self.ML_train_X),np.array(self.train_y))
            self.model_tree = clf_model
        return self.model_tree
    def get_model_lg(self,refresh=False):
        if self.model_lg == None or refresh == True:
            lg = LogisticRegression()
            lg_model = lg.fit(np.array(self.ML_train_X),np.array(self.train_y))
            self.model_lg = lg_model
        return self.model_lg
    def _predict(self,ML_X,model='lg'):
        prediction = None
        if model == "lg":
            prediction = self.get_model_lg().predict(np.array(ML_X))
        elif model == "tree":
            prediction = self.get_model_tree().predict(np.array(ML_X))
        return prediction
    def accuracy(self,ML_X,y,model = 'lg'):
        prediction = self._predict(ML_X,model)
        accuracy = np.mean(prediction==np.array(y))
        correct_idx = np.where(prediction==np.array(y))[0]
        incorrect_idx = np.where(prediction!=np.array(y))[0]
        print "accuracy:",accuracy
        print "correct_idx:",correct_idx
        print "incorrect_idx:",incorrect_idx 
    def checkOne(self,X,y,index,model='lg'):
        ML_X = self.classifier._make_ML_X([X[index]],print_out = True)
        print "predict class : ", self._predict(np.reshape(np.array(ML_X),(1,-1)),model)
        print "true class : ", y[index]
        print X[index]['message']

# Retail

In [1728]:
retail_ML = ML(['retail'])
print retail_ML.accuracy(retail_ML.ML_test_X, retail_ML.test_y,"lg")

accuracy: 0.84
correct_idx: [ 0  1  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 22 23 24 25 26 27
 28 29 30 31 32 34 35 36 37 38 39 40 41 42 43 47 48]
incorrect_idx: [ 2 20 21 33 44 45 46 49]
None


In [1729]:
retail_ML.checkOne(retail_ML.test_X,retail_ML.test_y,2,"lg")

match class_2 :  產品
match class_3 :  
match class_0 :  
match class_1 :  
predict class :  [2]
true class :  3
CITIZEN Watch Group展示廳

今年BASELWORLD的CITIZEN展場當中
有另一個亮點！－CITIZEN Watch Group的展示廳。

在這我們聚集
CITIZEN Watch Group現在所擁有的手錶品牌 ：
Alpina、Arnold & Son、BULOVA、
CAMPANOLA 、CITIZEN 以及 Ferderiue Constant .

在這廣大的空間内展示各品牌的最新產品，
不僅便於展現各自獨特的美學理念，
也能更好的與大眾溝通CITIZEN Watch Group的美好願景。

〉BASELWORLD 2017
 http://www.citizenwatch.com.tw/event2017/baselworld/index.html


# Household

In [1734]:
house_ML = ML(['Household & Personal Care'])
print house_ML.accuracy(house_ML.ML_test_X, house_ML.test_y,"lg")

accuracy: 0.96
correct_idx: [ 0  1  2  3  4  5  6  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48]
incorrect_idx: [ 7 49]
None


In [1737]:
house_ML.checkOne(house_ML.test_X,house_ML.test_y,7,"lg")

match class_2 :  試用
match class_3 :  活動
match class_0 :  得獎,好禮,優惠,得獎者
match class_1 :  加入
predict class :  [0]
true class :  1
【好奇寶寶聚樂部 「迎新入會尿布免費抽」- 🎉得獎名單🎉出爐囉！！】

歡迎新加入的媽咪們❤更恭喜這個月抽到✨五箱紫好奇✨的幸運兒，讓人好生羨慕啊😍😍😍~~。你也想把五箱紫好奇搬回家嗎？活動於4/30截止，還沒加入會員的媽咪們快來加入「好奇寶寶聚樂部」吧！
心動不如馬上行動🎁 立即加入好奇寶寶聚樂部: https://goo.gl/bhmiGO

專屬好奇寶寶聚樂部會員四大好禮: 
👏得到各項通路與產品的優惠資訊
👏新生寶寶專屬禮
👏產品試用包
👏孕期/育兒新知電子報

🎉 得獎者們記得在3/24(五)前主動聯絡我們唷，請來電0800-221-526或mail聯絡資訊至customerservice.0800@kcc.com！ 
聯絡資訊請提供Facebook姓名及登入用之Email、真實姓名、電話、收件地址，將由專人協助領獎。
得獎者恕不另行通知，逾期回覆將視同棄權，如有任何問題請於上班時間(週一至週五 09:00-12:00 / 13:30-17:00)洽詢好奇客服專線 0800-221 -526。


# Household + Retail & 不分keyword list (keyword_all.json)

In [1815]:
RetailHouse = ML(['Retail','Household & Personal Care'])
print RetailHouse.accuracy(RetailHouse.ML_test_X, RetailHouse.test_y,"lg")

accuracy: 0.9
correct_idx: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 27 28 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51
 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 72 73 74 75 76 77 79 80
 81 82 83 84 85 86 87 88 89 90 91 92 93 97 98]
incorrect_idx: [26 29 52 70 71 78 94 95 96 99]
None


In [1829]:
RetailHouse.checkOne(RetailHouse.test_X,RetailHouse.test_y,2,"lg")

match product_test :  
match product_new :  
match info_event :  
match monetary_incentive :  
match product_unit :  
match price_off :  75折
match engage_other :  
match product_version :  
match info_brand :  
match push_word :  
match engage_join :  
match engage_interaction :  
predict class :  [2]
true class :  2
💢明明很累卻睡不著
今晚又要失眠了嗎？😵
別沮喪！試試這招↓↓
💤睡前用晚安舒眠噴霧
對著枕頭、棉被、臥室
咻！咻！噴個幾下
馬上散發甜橙和薰衣草香氛🍊
整個人變得好放鬆
今晚一定好好睡！

晚安舒眠噴霧2瓶組↘75折
https://goo.gl/QanGwx

轉載自華人健康網

 — Products shown: 晚安舒眠噴霧2瓶組.


In [1830]:
np.bincount(RetailHouse.train_y)

array([16, 19, 36, 29])